In [1]:
import numpy as np
import open3d as o3d
from scipy.spatial.transform import Rotation as R
import pandas as pd

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def check_ray_rectangle_intersection(ray_origin, ray_direction, rectangle_corners):
    # Define the plane of the rectangle
    plane_normal = np.cross(rectangle_corners[1] - rectangle_corners[0], rectangle_corners[2] - rectangle_corners[0])
    plane_d = -np.dot(plane_normal, rectangle_corners[0])

    # Check if ray intersects the plane of the rectangle
    t = -(np.dot(plane_normal, ray_origin) + plane_d) / np.dot(plane_normal, ray_direction)
    if t < 0:
        return False, None  # Intersection is behind the ray origin

    # Calculate intersection point with the plane
    intersection_point = ray_origin + t * ray_direction

    # Check if intersection point is within rectangle boundaries
    min_x, max_x = min(rectangle_corners[:, 0]), max(rectangle_corners[:, 0])
    min_y, max_y = min(rectangle_corners[:, 1]), max(rectangle_corners[:, 1])
    min_z, max_z = min(rectangle_corners[:, 2]), max(rectangle_corners[:, 2])
    if not (min_x <= intersection_point[0] <= max_x and
            min_y <= intersection_point[1] <= max_y and
            min_z <= intersection_point[2] <= max_z):
        return False, None

    return True, intersection_point

In [3]:
def draw_geometries(pcds):
    """
    Draw Geometries
    Args:
        - pcds (): [pcd1,pcd2,...]
    """
    o3d.visualization.draw_geometries(pcds)


def get_o3d_FOR(origin=[0, 0, 0], size=10):
    """ 
    Create a FOR that can be added to the open3d point cloud
    """
    mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
        size=size)
    mesh_frame.translate(origin)
    return (mesh_frame)


def vector_magnitude(vec):
    """
    Calculates a vector's magnitude.
    Args:
        - vec (): 
    """
    magnitude = np.sqrt(np.sum(vec ** 2))
    return (magnitude)


def calculate_zy_rotation_for_arrow(vec):
    """
    Calculates the rotations required to go from the vector vec to the 
    z axis vector of the original FOR. The first rotation that is 
    calculated is over the z axis. This will leave the vector vec on the
    XZ plane. Then, the rotation over the y axis. 

    Returns the angles of rotation over axis z and y required to
    get the vector vec into the same orientation as axis z
    of the original FOR

    Args:
        - vec (): 
    """
    # Rotation over z axis of the FOR
    gamma = np.arctan(vec[1] / vec[0])
    Rz = np.array([[np.cos(gamma), -np.sin(gamma), 0],
                   [np.sin(gamma), np.cos(gamma), 0],
                   [0, 0, 1]])
    # Rotate vec to calculate next rotation
    vec = Rz.T @ vec.reshape(-1, 1)
    vec = vec.reshape(-1)
    # Rotation over y axis of the FOR
    beta = np.arctan(vec[0] / vec[2])
    Ry = np.array([[np.cos(beta), 0, np.sin(beta)],
                   [0, 1, 0],
                   [-np.sin(beta), 0, np.cos(beta)]])
    return (Rz, Ry)


def create_arrow(scale=10):
    """
    Create an arrow in for Open3D
    """
    cone_height = scale * 0.2
    cylinder_height = scale * 0.8
    cone_radius = scale / 10
    cylinder_radius = scale / 20
    mesh_frame = o3d.geometry.TriangleMesh.create_arrow(cone_radius=1,
                                                        cone_height=cone_height,
                                                        cylinder_radius=0.5,
                                                        cylinder_height=cylinder_height)
    return (mesh_frame)


def get_arrow(origin=[0, 0, 0], end=None, vec=None) -> o3d.geometry.TriangleMesh:
    """
    Creates an arrow from an origin point to an end point,
    or create an arrow from a vector vec starting from origin.
    Args:
        - end (): End point. [x,y,z]
        - vec (): Vector. [i,j,k]
    """
    scale = 10
    Ry = Rz = np.eye(3)
    T = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
    T[:3, -1] = origin
    if end is not None:
        vec = np.array(end) - np.array(origin)
    elif vec is not None:
        vec = np.array(vec)
    if end is not None or vec is not None:
        scale = vector_magnitude(vec)
        Rz, Ry = calculate_zy_rotation_for_arrow(vec)
    mesh = create_arrow(scale)
    # Create the arrow
    mesh.rotate(Ry, center=np.array([0, 0, 0]))
    mesh.rotate(Rz, center=np.array([0, 0, 0]))
    mesh.translate(origin)
    return (mesh)

In [4]:
def get_rectangle_corners(x, y, z, yaw, pitch, roll, width, height):
    # Convert angles from degrees to radians
    yaw_rad = np.deg2rad(yaw)
    pitch_rad = np.deg2rad(pitch)
    roll_rad = np.deg2rad(roll)

    # Create rotation matrix from Euler angles
    rotation_matrix = R.from_euler('zyx', [yaw_rad, pitch_rad, roll_rad]).as_matrix()

    # Define the four corners of the rectangle in local coordinates
    half_width = width / 2
    half_height = height / 2
    corners_local = np.array([
        [-half_width, -half_height, 0],
        [half_width, -half_height, 0],
        [half_width, half_height, 0],
        [-half_width, half_height, 0]
    ])

    # Apply rotation and translation to get the corners in global coordinates
    corners_global = (rotation_matrix @ corners_local.T).T + np.array([x, y, z])

    return corners_global


In [5]:
field_w, field_h = (54 * 12) + 3.25, (26 * 12) + 11.25

width, height = 41.375000, 20.119292  # Size
x, y, z = 84.111, 78.136139, field_w  # Position, bottom left corner, RED ALLIANCE

# Convert x, y to centre
x = x + width / 2
y = y + height / 2

yaw, pitch, roll = 0, 0, 104.000000  # Orientation in degrees

corners = get_rectangle_corners(x, y, z, yaw, pitch, roll, width, height)

In [44]:
num_shots = 5000
rays = []
raw_rays = []
for _ in range(0, num_shots):
    # height = np.random.uniform(0, 10)
    height = 0

    fy, fx = np.random.uniform(0, field_h), np.random.uniform(0, field_w)

    # Generate a random shot
    ray_origin = np.array([fy, height, fx])

    ray_direction = np.array([np.random.uniform(-1, 1), np.random.uniform(0, 1), np.random.uniform(0, 1)])

    intersect, intersection = check_ray_rectangle_intersection(ray_origin, ray_direction, corners)

    raw_rays.append([ray_origin, ray_direction, intersect, intersection])
    arrow = get_arrow(ray_origin, vec=ray_direction * 1000)
    if intersect:
        arrow.paint_uniform_color([1, 0, 0])
        rays.append(arrow)
    else:
        if np.random.uniform(0, 1) > 0.95:
            rays.append(arrow)

In [ ]:
vislualise = True

if vislualise:
    # Create a Cartesian Frame of Reference
    FOR = get_o3d_FOR()

    # Create a PointCloud object
    pcd = o3d.geometry.PointCloud()

    corners_o3d = o3d.utility.Vector3dVector(corners)

    # Convert numpy array to open3d format
    pcd.points = corners_o3d

    # Define the lines that form the rectangle
    lines = [[0, 1], [1, 2], [2, 3], [3, 0]]

    # Create a LineSetray_origin = np.array([0, 0, 0])
    line_set = o3d.geometry.LineSet()

    # Set the points and lines of the line set
    line_set.points = o3d.utility.Vector3dVector(corners)
    line_set.lines = o3d.utility.Vector2iVector(lines)

    # intersected, intersection_point = check_ray_rectangle_intersection(ray_origin, ray_direction, corners)
    # print(f"Intersected: {intersected}, Intersection point: {intersection_point}")

    o3d.visualization.draw_geometries([FOR, pcd, line_set] + rays)

In [368]:
data_pd = pd.DataFrame([],
                       columns=['time', 'theta', 'shooter_speed', 'pivot_angle', 'elevator_height', 'x', 'y', 'vx',
                                'vy', 'went_in'])

In [18]:
def ray_direction_to_angle(direction):
    # Extract the x and z components of the ray direction

    # Calculate the angle in radians
    angle_rad = np.arctan2(direction[2], direction[0])

    # Convert the angle to degrees
    angle_deg = np.rad2deg(angle_rad)

    return 90 - angle_deg


def ray_direction_to_pivot_angle(direction):
    # Extract the x and z components of the ray direction

    # Calculate the angle in radians
    angle_rad = np.arctan2(direction[1], direction[2])

    # Convert the angle to degrees
    angle_deg = np.rad2deg(angle_rad)

    if angle_deg < 0:
        angle_deg = 360 + angle_deg

    return angle_deg

In [371]:
for i, ray in enumerate(raw_rays):
    data_pd.loc[i] = [i, ray_direction_to_angle(ray[1]), 5000, ray_direction_to_pivot_angle(ray[1]), 0, ray[0][2],
                      ray[0][0], 0, 0, (1 if ray[2] else 0)]

In [372]:
data_pd.to_csv("../data/raycasting.csv", index=False)